In [1]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import env

# DBSCAN import
from sklearn.cluster import DBSCAN
# Scaler import
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None,
                    names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])

In [3]:
df

,date,time,page,id,cohort,ip
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61
...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86


In [4]:
def get_connection(db, user=env.user, host=env.host, password=env.password):
        return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [5]:
df2 = pd.read_sql('''            
                            SELECT id as cohort, start_date, end_date, program_id 
                            FROM curriculum_logs.cohorts;  ''', get_connection('curriculum_logs'))

In [6]:
df2

,cohort,start_date,end_date,program_id
0,1,2014-02-04,2014-04-22,1
1,2,2014-06-04,2014-08-22,1
2,3,2014-09-04,2014-11-05,1
3,4,2014-10-20,2015-01-18,1
4,5,2014-11-18,2015-02-24,1
5,6,2015-02-03,2015-05-26,1
6,7,2015-06-05,2015-10-06,1
7,8,2015-09-22,2016-02-06,1
8,9,2015-03-30,2015-07-29,4
9,10,2015-11-03,2016-03-11,4


In [7]:
logs = pd.merge(df, df2, how='left', left_on='cohort', right_on='cohort')
logs

,date,time,page,id,cohort,ip,start_date,end_date,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-08,2018-05-17,2.0
...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,2014-02-04,2014-02-04,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,2014-02-04,2014-02-04,2.0


In [9]:
(logs['date'] >= logs.start_date) & (logs['date'] <= logs.end_date)


0         False
1         False
2         False
3         False
4          True
          ...  
900218    False
900219    False
900220    False
900221    False
900222    False
Length: 900223, dtype: bool

In [11]:
logs['active'] = np.where((logs['date'] >= logs.start_date) & (logs['date'] <= logs.end_date), 1, 0)
logs.active.value_counts()


active
1    647496
0    252727
Name: count, dtype: int64

In [12]:
hi

NameError: name 'hi' is not defined